##### Grupo 10: Belén Sánchez Centeno y Martín Fernández de Diego

In [1]:
cd ../aima-python

/Users/martin/Documents/Cursos/2020-2021/IA/Laboratorio/Python/aima-python


In [2]:
import search
from search import Problem
from search import breadth_first_tree_search, breadth_first_graph_search, depth_first_tree_search, depth_first_graph_search, astar_search

# Problema 1.
Un grupo de 5 personas quiere cruzar un viejo y estrecho puente. Es una noche cerrada y se necesita llevar una linterna para cruzar. El grupo solo dispone de una linterna, a la que le quedan 5 minutos de batería.
1. Cada persona tarda en cruzar 10, 30, 60, 80 y 120 segundos, respectivamente.
2. El puente solo resiste un máximo de 2 personas cruzando a la vez, y cuando cruzan dos personas juntas, caminan a la velocidad del más lento.
3. No se puede lanzar la linterna de un extremo a otro del puente, así que
cada vez que crucen dos personas, alguien tiene que volver a cruzar hacia atrás con la linterna a buscar a los compañeros que falten, y así hasta que hayan cruzado todos.

### Representación del estado
Estado: (Posición de cada persona respecto del puente, (Posición de la linterna respecto del puente, Batería restante))  
Orilla inicial := 0  
Orilla final := 1

In [3]:
# Variables globales
battery = 300
cost = (10,30,60,80,120)

In [4]:
# Creamos la clase
class ProblemaLinterna(Problem):
    '''Clase problema siguiendo la estructura que AIMA espera que tengan los problemas.'''
    def __init__(self, initial = (0,0,0,0,0,(0,battery)), goal = None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)

    def actions(self, state):
        '''Devuelve las acciones validas para un estado.'''
        action = list()
        for i in range(0,5):
            if self._is_valid(state,i):
                action.append(("Cruza 1",i))
        for i in range(0,5):
            for j in range(i+1,5):
                if self._is_valid(state,i) and self._is_valid(state,j):
                    action.append(("Cruzan 2",(i,j)))
        return action

    def result(self, state, action):
        '''Devuelve el estado resultante de aplicar una accion a un estado.'''
        people = list(state[0:5])
        lantern = list(state[5])
        lantern[0] = 1 - state[5][0] # Movemos linterna
        if action[0] == "Cruza 1":
            people[action[1]] = 1 - state[action[1]] # Movemos persona
            lantern[1] = state[5][1] - cost[action[1]] # Actualizamos la batería
        else:
            people[action[1][0]] = 1 - state[action[1][0]] # Movemos persona1
            people[action[1][1]] = 1 - state[action[1][1]] # Movemos persona2
            lantern[1] = state[5][1] - max(cost[action[1][0]],cost[action[1][0]]) # Actualizamos la batería
        people.append(tuple(lantern))
        return tuple(people)
    
    def goal_test(self, state):
        '''Determina si el estado ha alcanzado el estado objetivo.'''
        person = True
        for i in range(0,5):
            person = person and state[i] == 1
        return person and state[5][0] == 1 # and estado[5][1] >= 0
    
    def path_cost(self, c, state1, action, state2):
        '''Devuelve el coste de la solución tras aplicar la acción.'''
        return c + self._costs(action)
        # La redefinición de la función path_cost refina la solución final
        
    def _is_valid(self, state, i):
        '''Determina si un estado es valido.'''
        return state[5][0] == state[i] and state[5][1] - cost[i] >= 0 
    
    def _costs(self, action):
        '''Devuelve el coste de aplicar accion a estado.'''
        if action[0] == "Cruza 1":
            return cost[action[1]]
        else:
            return max(cost[action[1][0]],cost[action[1][1]])

In [5]:
def HeuristicaLinterna(node):
    '''Calcula el coste restante en el nodo si fuera de día.'''
    # Eliminamos la restricción de la linterna
    state = node.state
    total = 0;
    # Aseguramos que las personas con mayor coste vayan juntas
    i = 4;
    while i >= 0:
        if state[i] == 0:
            total += cost[i]
            # Buscamos la siguiente persona con mayor coste
            while i > 0:
                i -= 1
                if state[i] == 0:
                    break
        i -= 1
    return total

### Resultados previos

In [6]:
pLinterna = ProblemaLinterna()

In [7]:
pLinterna.initial

(0, 0, 0, 0, 0, (0, 300))

In [8]:
pLinterna.actions(pLinterna.initial)

[('Cruza 1', 0),
 ('Cruza 1', 1),
 ('Cruza 1', 2),
 ('Cruza 1', 3),
 ('Cruza 1', 4),
 ('Cruzan 2', (0, 1)),
 ('Cruzan 2', (0, 2)),
 ('Cruzan 2', (0, 3)),
 ('Cruzan 2', (0, 4)),
 ('Cruzan 2', (1, 2)),
 ('Cruzan 2', (1, 3)),
 ('Cruzan 2', (1, 4)),
 ('Cruzan 2', (2, 3)),
 ('Cruzan 2', (2, 4)),
 ('Cruzan 2', (3, 4))]

In [9]:
pLinterna.result(pLinterna.initial,("Cruza 1",0))

(1, 0, 0, 0, 0, (1, 290))

In [10]:
pLinterna.goal_test(pLinterna.initial)

False

### Búsquedas de solución

In [11]:
breadth_first_graph_search(pLinterna).solution()

[('Cruzan 2', (0, 1)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 2)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 3)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 4))]

In [12]:
breadth_first_graph_search(pLinterna).path()

[<Node (0, 0, 0, 0, 0, (0, 300))>,
 <Node (1, 1, 0, 0, 0, (1, 290))>,
 <Node (0, 1, 0, 0, 0, (0, 280))>,
 <Node (1, 1, 1, 0, 0, (1, 270))>,
 <Node (0, 1, 1, 0, 0, (0, 260))>,
 <Node (1, 1, 1, 1, 0, (1, 250))>,
 <Node (0, 1, 1, 1, 0, (0, 240))>,
 <Node (1, 1, 1, 1, 1, (1, 230))>]

In [13]:
%%timeit
breadth_first_graph_search(pLinterna).solution()

142 ms ± 6.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
depth_first_graph_search(pLinterna).solution()

[('Cruzan 2', (3, 4)),
 ('Cruzan 2', (3, 4)),
 ('Cruzan 2', (1, 4)),
 ('Cruza 1', 1),
 ('Cruzan 2', (0, 3)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 2)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 1))]

In [15]:
depth_first_graph_search(pLinterna).path()

[<Node (0, 0, 0, 0, 0, (0, 300))>,
 <Node (0, 0, 0, 1, 1, (1, 220))>,
 <Node (0, 0, 0, 0, 0, (0, 140))>,
 <Node (0, 1, 0, 0, 1, (1, 110))>,
 <Node (0, 0, 0, 0, 1, (0, 80))>,
 <Node (1, 0, 0, 1, 1, (1, 70))>,
 <Node (0, 0, 0, 1, 1, (0, 60))>,
 <Node (1, 0, 1, 1, 1, (1, 50))>,
 <Node (0, 0, 1, 1, 1, (0, 40))>,
 <Node (1, 1, 1, 1, 1, (1, 30))>]

In [16]:
%%timeit
depth_first_graph_search(pLinterna).solution()

753 µs ± 41.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
astar_search(pLinterna,HeuristicaLinterna).solution()

[('Cruzan 2', (0, 2)),
 ('Cruza 1', 0),
 ('Cruzan 2', (0, 1)),
 ('Cruza 1', 0),
 ('Cruzan 2', (3, 4)),
 ('Cruza 1', 1),
 ('Cruzan 2', (0, 1))]

In [18]:
astar_search(pLinterna,HeuristicaLinterna).path()

[<Node (0, 0, 0, 0, 0, (0, 300))>,
 <Node (1, 0, 1, 0, 0, (1, 290))>,
 <Node (0, 0, 1, 0, 0, (0, 280))>,
 <Node (1, 1, 1, 0, 0, (1, 270))>,
 <Node (0, 1, 1, 0, 0, (0, 260))>,
 <Node (0, 1, 1, 1, 1, (1, 180))>,
 <Node (0, 0, 1, 1, 1, (0, 150))>,
 <Node (1, 1, 1, 1, 1, (1, 140))>]

In [19]:
%%timeit
astar_search(pLinterna,HeuristicaLinterna).solution()

115 ms ± 5.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Conclusiones
Vistos los resultados previos,  
En cuanto a **mejores soluciones**, podemos concluir que las arrojan el algoritmo *breadth_first_graph_search* y *astar_search* porque ambas consiguen llegar al estado objetivo en 7 acciones. Hay que resaltar que *breadth_first_graph_search* es aún mejor porque alcanza la solución del problema con 230 segundos de batería restante en la linterna mientras que *astar_search* alcanza la solución con 140 segundos.  
En cuanto a **mejores tiempos**, podemos concluir que los arroja el algoritmo *depth_first_graph_search*. En cambio, su solución no es muy buena.  

Por tanto, si queremos una solución con un buen rendimiento, podríamos tomar la dada por *astar_search* ya que tarda un poco menos que *breadth_first_graph_search*. Sin embargo, esta última, consigue una solución mucho mejor con un tiempo no muy alejado del de *astar_search*.  

Bastaría mejorar ligeramente la heurística *HeuristicaLinterna* para mejorar el rendimiento de la solución mediante el algoritmo *astar_search*.

# Problema 4.
Grados de separación entre actores. Este problema es de naturaleza algo diferente ya que tendréis que trabajar con un grafo de datos sobre actores y películas. Según la teoría de los grados de separación de Kevin Bacon cualquier persona de la industria del cine de Hollywood puede conectarse con Kevin Bacon en un máximo de 6 pasos donde cada paso es una película en la que han participado dos personas. En este problema os pedimos encontrar el camino más corto entre dos actores dados, a través de la secuencia de películas que los conectan. Por ejemplo, el camino más corto entre Jennifer Lawrence y Tom Hanks tiene longitud 2: Jennifer Lawrence se conecta con Kevin Bacon por la película “X-Men: First Class,” y Kevin Bacon con Tom Hanks por “Apollo 13.”  

Podemos definir este problema como un problema de búsqueda en el que los estados son las personas y las acciones son las películas que nos llevan de un actor al siguiente. El estado inicial y final son las personas que queremos conectar. Resuelve el problema indicando qué algoritmo es el más adecuado.  

Se os proporciona la base de datos en formato csv con actores que participan en películas de dos tamaños (pequeño y grande en dos directorios) y el código Python para cargar los datos a través de estructuras diccionario. Se puede consultar una persona por su nombre (y obtener su id) y las películas en las que ha participado. La función load_data carga los archivos csv en los diccionarios.  

En el fichero de personas (people.csv) cada persona tiene un id único, un nombre y su año de nacimiento (102,"Kevin Bacon",1958). Y en el archivo de películas (movies.csv) cada película tiene también un id único, su título y el año (109830,"Forrest Gump",1994). En el tercer archivo (stars.csv) se establece la relación entre las personas y las películas. La persona con id 102 participó en la película con id 104257.  

En primer lugar, debes descargar y entender el contenido de los archivos y estructuras de datos y el código que os damos ya hecho para leer los datos, obtener el id dado el nombre y obtener las conexiones de una persona dada. Dado el id de una persona, la función neighbors_for_person devuelve el conjunto de pares (movie_id, person_id) con todas las personas que participaron con esa persona en la película con movie_id. Si hay un camino entre pi y po, el camino entre ellas puede ser devuelto como una lista de pares de enteros (movie_id, person_id). Por ejemplo, si el camino más corto es: [(1,2), (3,4)], significa que pi participó en la película 1 con la persona 2, la persona 2 participó en la película 3 con la persona 4. Y como termina el camino, 4 es po.

#### Ejemplo de salida:
Loading data...  
Name: Emma Watson  
Name: Jennifer Lawrence  
3 grados de separación  
1: Emma Watson y Brendan Gleeson participaron en Harry Potter and the Order of the Phoenix  
2: Brendan Gleeson y Michael Fassbender participaron en Trespass Against Us  
3: Michael Fassbender y Jennifer Lawrence participaron en X-Men: First Class

Si hay varios caminos de la misma longitud se puede devolver cualquiera o puedes proponer algún criterio adicional (por ejemplo, usando el año). Puede ocurrir que no haya camino entre dos actores.

In [20]:
cd ../P1/grados

/Users/martin/Documents/Cursos/2020-2021/IA/Laboratorio/Python/P1/grados


In [21]:
## grados.py
import csv
import sys

# dictionary of people names with ids 
names = {}
# dictionary: name, birth, movies (a set of movie_ids)
people = {}
# movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Loading data from people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Loading data from movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Loading data from stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Para ejecutarlo en línea de comandos: python grados.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Cargando los datos...")
    load_data("large")

    source = person_id_for_name(input("Nombre: "))
    if source is None:
        sys.exit("Esa persona no se encuentra.")
    target = person_id_for_name(input("Nombre: "))
    if target is None:
        sys.exit("Esa persona no se encuentra.")

    print("Analizando los datos...")
    path = shortest_path(source, target)

    if path is None:
        print("No están conectados.")
    else:
        degrees = len(path)
        print(f"{degrees} grados de separacion.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} y {person2} participaron en {movie}")


def shortest_path(source, target):
    """
    Devuelve la lista de pares (movie_id, person_id) que conectan source y target o None si no hay conexion.
    """
    if (source == target):
        return list()
    
    # Utilizamos BFS para hallar el camino más corto
    marked = list()
    queue = list()
    edgeTo = {}
    
    marked.append(source)
    queue.append(source)
    
    stop = False
    while len(queue) > 0 and not stop:
        person1 = queue.pop(0) # Desapilamos el primer elemento de la cola
        for (movie,person2) in neighbors_for_person(person1):
            if not person2 in marked:
                marked.append(person2)
                edgeTo[person2] = (movie,person1)
                queue.append(person2) # Apilamos por el final de la cola
                if person2 == target:
                    stop = True
                    break
    
    # Reconstruimos el camino
    if target in marked:
        path = list()
        person = target
        while person != source:
            (movie,personAux) = edgeTo[person]
            path.append((movie,person))
            person = personAux
        path.reverse()
        return path
    else:
        return None

    
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name, resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [22]:
def Kevin_Bacon():
    print("Cargando los datos...")
    load_data("large")

    source = person_id_for_name(input("Nombre del actor fuente: "))
    if source is None:
        sys.exit("Esa persona no se encuentra.")
    target = person_id_for_name(input("Nombre del actor objetivo: "))
    if target is None:
        sys.exit("Esa persona no se encuentra.")

    print("Analizando los datos...")
    path = shortest_path(source, target)

    if path is None:
        print("No están conectados.")
    elif len(path) == 0:
        print("Son el mismo actor.")
    else:
        degrees = len(path)
        print(f"{degrees} grados de separacion.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} y {person2} participaron en {movie}")

In [23]:
Kevin_Bacon()

Cargando los datos...
Nombre del actor fuente: Bill Murray
Nombre del actor objetivo: Dustin Hoffman
Analizando los datos...
2 grados de separacion.
1: Bill Murray y Meryl Streep participaron en Fantastic Mr. Fox
2: Meryl Streep y Dustin Hoffman participaron en Kramer vs. Kramer
